In [2]:
import json
import pandas as pd
import glob
import os

In [ ]:
cnt = 0
total_length_mean = 0
for annotation in glob.glob('./induction_data/annotations/*.json'):
    with open(annotation, 'r') as task_f:
        s = task_f.read()
        task_data = json.loads(s)
    task_name = annotation.split('/')[-1].split('.')[0]

    input_prefix = './induction_data/induction_input'
    cnt += len(task_data['annotations'])
    length_mean = 0
    for i, task_definition in enumerate(task_data['annotations']):
        length_mean += task_definition.split().__len__()
        
        task_path = input_prefix + '/' + task_name + '.json'
        with open(task_path, 'r') as task_f:
            s = task_f.read()
            task_data_natural = json.loads(s)
            task_data_natural['Definition'] = [task_definition]
            task_data_natural["Task"] = task_name
            task_data_natural["Contributors"] = 'instruction_induction'
            task_data_natural["Source"] = ['instruction_induction']
            task_data_natural["URL"] = ['instruction_induction']
            task_data_natural["Categories"] = ['instruction_induction']
            task_data_natural["Reasoning"] = ['instruction_induction']
            task_data_natural["Positive Examples"] = []
            task_data_natural["Negative Examples"] = []
            task_data_natural["Input_language"] = ['English']
            task_data_natural["Output_language"] = ['English']
            task_data_natural["Instruction_language"] = ['English']
            task_data_natural["Domains"] = ['instruction_induction']
            task_data_natural["Instances"] = []
            all_instances = task_data_natural.pop("examples")
            all_instances = list(all_instances.values())
            for idx, instance in enumerate(all_instances):
                example = task_data.copy()
                input_output = instance['input']
                for pair in input_output.split('\n\n'):
                    if pair.startswith('Input:'):
                        input_ = pair.split('\n')[0].strip('Input: ')
                        output_ = pair.split('\n')[1].strip('Output: ')
                _instance = {'id': hash(input_output),'input': input_, 'output': [output_]}
                task_data_natural["Instances"].append(_instance)
            
            task_data_natural["Instance License"] = ["Unknown"]
            task_prefix = './induction_data/tasks'
            task_path = task_prefix + '/' + task_name + '_' + str(i) + '.' + 'json'
            print(task_data_natural)
            with open(task_path, 'w') as _task_f:
                json.dump(task_data_natural, _task_f, indent=2)
        
    total_length_mean += length_mean / len(task_data['annotations'])
        # break
print(total_length_mean/24)
print(cnt/24)

In [1]:
import json
import collections
import pandas as pd
import numpy as np
import glob

task_nums = {}
for annotation in glob.glob('./induction_data/annotations/*.json'):
    with open(annotation, 'r') as task_f:
        s = task_f.read()
        task_data = json.loads(s)
    task_name = annotation.split('/')[-1].split('.')[0]
    task_nums[task_name] = len(task_data['annotations'])

exact_match = collections.defaultdict(list)
rougeL = collections.defaultdict(list)


metric_file = open('./induction_outputs/output_xlingual_induction_0/all_results.json', 'r')
metric = json.load(metric_file)
metric_names = task_nums.keys()
def create_dict(keys):
    d = {}
    for key in keys:
        d[key] = []
    return d    
for task_name, num in task_nums.items():
    metric_samples = collections.defaultdict(list)
    metric_mean = 0
    metric_std = 0
    for index in range(num):
            metric_samples[task_name].append(metric[f'eval_exact_match_for_{task_name}_{index}'])
    
    metric_mean = round(np.mean(metric_samples[task_name]), 1)
    metric_std = round(np.std(metric_samples[task_name]), 1)
        
    exact_match[task_name].append(str(metric_mean) + "\u00B1" + str(metric_std))

    metric_samples = collections.defaultdict(list)
    metric_mean = 0
    metric_std = 0
    for index in range(num):
            metric_samples[task_name].append(metric[f'eval_rougeL_for_{task_name}_{index}'])
    
    metric_mean = round(np.mean(metric_samples[task_name]), 1)
    metric_std = round(np.std(metric_samples[task_name]), 1)
        
    rougeL[task_name].append(str(metric_mean) + "\u00B1" + str(metric_std))
    


In [2]:
induction_metric = pd.concat([pd.DataFrame(exact_match), pd.DataFrame(rougeL)])
induction_metric.index = ['exact_match', 'rougeL']

In [3]:
exact_match

defaultdict(list,
            {'translation_en-es': ['8.5±0.5'],
             'translation_en-de': ['7.7±0.5'],
             'sum': ['0.0±0.0'],
             'synonyms': ['1.0±0.0'],
             'informal_to_formal': ['0.0±0.0'],
             'negation': ['0.0±0.0'],
             'rhymes': ['1.0±0.0'],
             'orthography_starts_with': ['1.2±0.4'],
             'cause_and_effect': ['26.9±10.9'],
             'taxonomy_animal': ['0.0±0.0'],
             'larger_animal': ['5.0±2.5'],
             'diff': ['1.1±0.9'],
             'common_concept': ['0.0±0.0'],
             'first_word_letter': ['1.0±0.0'],
             'antonyms': ['13.6±1.6'],
             'word_in_context': ['9.0±12.2'],
             'translation_en-fr': ['9.2±0.8'],
             'letters_list': ['2.0±0.0'],
             'active_to_passive': ['0.4±0.7'],
             'second_word_letter': ['1.3±0.5'],
             'singular_to_plural': ['0.8±0.4'],
             'sentence_similarity': ['0.0±0.0'],
             's

In [5]:
print(induction_metric.T.to_latex())

\begin{tabular}{lll}
\toprule
{} & exact\_match &     rougeL \\
\midrule
translation\_en-es       &     8.5±0.5 &    8.5±0.5 \\
translation\_en-de       &     7.7±0.5 &    9.5±1.0 \\
sum                     &     0.0±0.0 &    0.0±0.0 \\
synonyms                &     1.0±0.0 &    4.3±0.4 \\
informal\_to\_formal      &     0.0±0.0 &   30.3±0.4 \\
negation                &     0.0±0.0 &  70.7±11.6 \\
rhymes                  &     1.0±0.0 &    1.0±0.0 \\
orthography\_starts\_with &     1.2±0.4 &   18.1±2.5 \\
cause\_and\_effect        &   26.9±10.9 &  50.5±15.1 \\
taxonomy\_animal         &     0.0±0.0 &   24.0±3.9 \\
larger\_animal           &     5.0±2.5 &   20.1±8.0 \\
diff                    &     1.1±0.9 &    1.6±0.5 \\
common\_concept          &     0.0±0.0 &    0.0±0.0 \\
first\_word\_letter       &     1.0±0.0 &    1.0±0.0 \\
antonyms                &    13.6±1.6 &   14.3±1.3 \\
word\_in\_context         &    9.0±12.2 &  19.5±21.7 \\
translation\_en-fr       &     9.2±0.8 &   12.4±

/tmp/ipykernel_22729/634863946.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(induction_metric.T.to_latex())


In [23]:
induction_metric.T

,exact_match,rougeL
translation_en-es,8.5±0.5,8.5±0.5
translation_en-de,7.7±0.5,9.5±1.0
sum,0.0±0.0,0.0±0.0
synonyms,1.0±0.0,4.3±0.4
informal_to_formal,0.0±0.0,30.3±0.4
negation,0.0±0.0,70.7±11.6
rhymes,1.0±0.0,1.0±0.0
orthography_starts_with,1.2±0.4,18.1±2.5
cause_and_effect,26.9±10.9,50.5±15.1
taxonomy_animal,0.0±0.0,24.0±3.9
